## 会话摘要记忆 summary
ConversationSummaryMemory记忆类型会随着时间的推移对对话进行摘要。这对于从对话中压缩信息非常有用。会话摘要记忆会即时总结对话并将当前摘要存储在记忆中。  
这种记忆对于较长的对话非常有用，因为将过去的消息历史原样保留在提示中会占用太多的token。
### LangChain老版本

In [1]:
# from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
        # model="Qwen/Qwen2.5-72B-Instruct",
        model="Qwen/Qwen3-8B",
        # 硅基流动
        api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
        base_url="https://api.siliconflow.cn/v1/",
        # # modelscope 
        # base_url="https://api-inference.modelscope.cn/v1/",
        # api_key="ms-e2666046-2f3b-4c76-bcc0-e21f8ebf9ea1",
    )

In [ ]:
memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

### LangGraph新版本

In [2]:
from typing import Any, TypedDict

from langchain_core.messages import AnyMessage
from langchain_core.messages.utils import count_tokens_approximately
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.checkpoint.memory import InMemorySaver
from langmem.short_term import SummarizationNode, RunningSummary  

model = llm
summarization_model = model.bind(max_tokens=128)

class State(MessagesState):
    context: dict[str, RunningSummary]  

class LLMInputState(TypedDict):  
    summarized_messages: list[AnyMessage]
    context: dict[str, RunningSummary]

summarization_node = SummarizationNode(  
    token_counter=count_tokens_approximately,
    model=summarization_model,
    max_tokens=256,
    max_tokens_before_summary=256,
    max_summary_tokens=128,
)

def call_model(state: LLMInputState):  
    response = model.invoke(state["summarized_messages"])
    return {"messages": [response]}

checkpointer = InMemorySaver()
builder = StateGraph(State)
builder.add_node(call_model)
builder.add_node("summarize", summarization_node)  
builder.add_edge(START, "summarize")
builder.add_edge("summarize", "call_model")
graph = builder.compile(checkpointer=checkpointer)

# Invoke the graph
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": "你好，我的名字是鲍勃"}, config)
graph.invoke({"messages": "写一首关于猫的短诗"}, config)
graph.invoke({"messages": "现在为狗写一首类似的诗"}, config)
# 添加对长话消息的支持
graph.invoke({"messages": "我最近在学习关于LangChain的内容，你能帮我总结一下吗？"}, config)
graph.invoke({"messages": "请介绍杨振宁先生的生平事迹"}, config)
graph.invoke({"messages": "他在物理学领域有哪些重要的贡献？"}, config)
graph.invoke({"messages": "他的最杰出的工作是什么？"}, config)
graph.invoke({"messages": "他和中国东北大学有什么关系？"}, config)
final_response = graph.invoke({"messages": "我的名字是什么？"}, config)

final_response["messages"][-1].pretty_print()
print("\nSummary:", final_response["context"]["running_summary"].summary)

/Users/lzc/miniconda3/envs/deepresearch/lib/python3.11/site-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(
/Users/lzc/miniconda3/envs/deepresearch/lib/python3.11/site-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(
/Users/lzc/miniconda3/envs/deepresearch/lib/python3.11/site-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warning

================================== Ai Message ==================================



您好，我是通义千问。请问您叫什么名字呢？

Summary: 

---

### **扩展总结：科学、人文与技术的三位一体（深化版）**
在原有对话框架的基础上，进一步深化科学、人文与技术三者之间的内在联系，揭示杨振宁先生的学术遗产如何为当代知识实践提供多重启发：

#### **1. 科学理论与人文精神的共生（文化与理性的交融）**
杨振宁的科学成就不仅体现在公式推导与实验验证中，还深刻反映了**文化语境与理性思维的相互塑造**：  
- **诗意化的科学表达**：他的研究常常以“诗性逻辑”为灵感，例如通过汉字结构、古典诗词的对仗与隐喻，理解物理规律的对称性与不平衡性。这种思维方式在现代AI开发中同样重要，LangChain通过注释、模块化设计和“链”的逻辑，实现了技术语言与人文表达的有机融合。  
- **历史叙事与科学突破**：杨振宁始终强调科学史的“脉络性”，认为突破需要站在巨人的肩膀上。这一理念映射到LangChain的学习中，技术的迭代性需要理解其历史背景（如RAG的演变、工具调用的兴起），而非孤立地追求代码优化。例如，“缓存”与“状态管理”功能背后，正是对“旧知识”与“新问题”之间动态关系的抽象化处理。

#### **2. 技术工具链与跨学科思维的镜像（解构与重构的实践）**
杨振宁的“非对称性”与“未知领域”探索，为技术工具链的设计提供了**方法论层面的镜像**：  
- **从物理对称到技术解耦**：杨-米尔斯理论通过破除电磁学的对称性，建立了描述强弱相互作用的新体系。这种“破旧立新”的模式，与LangChain通过链式结构实现不同模块（如RAG、工具调用）之间的解耦与协作高度相似。技术学习者需掌握这种“分而治之”的逻辑，以应对复杂系统的模块化需求。  
- **隐喻思维在技术中的应用**：杨振宁对“道”的哲学思考（如“无穷大”与“无限小”的辩证关系）可类比为技术中的“图层化视角”。例如，在LangChain中，通过分层结构处理代码与模型的交互（如链、代理、记忆组件），本质上是对物理世界中“局部规则”与“全局观察”的模拟。这种隐喻思维能帮助学习者跨越技术实现的边界，聚焦于功能本质

In [3]:
count_tokens_approximately(final_response["messages"])

2566

In [4]:
final_response

{'messages': [HumanMessage(content='你好，我的名字是鲍勃', additional_kwargs={}, response_metadata={}, id='261ddc85-fedd-456a-83e2-6e2bf1b3e868'),
  AIMessage(content='\n\n你好，鲍勃！很高兴认识你😊 你是想聊聊天，还是有具体的问题需要解答呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 238, 'prompt_tokens': 15, 'total_tokens': 253, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 214, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019a0a164c332dac2d039dde81551d04', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--5076e986-6845-4d65-8e87-be97022f0bc3-0', usage_metadata={'input_tokens': 15, 'output_tokens': 238, 'total_tokens': 253, 'input_token_details': {}, 'output_token_details': {'reasoning': 214}}),
  HumanMessage(content='写一首关于猫的短诗', additional_kwargs={}, response_metadata={}, id='0e00fca6-71ab-44ef-9d3f-272

In [6]:
count_tokens_approximately(final_response["context"]["running_summary"].summary)


17965

存在的问题，summary的token长度超过上下文Message的token长度。